In [1]:
import numpy as np
import pandas as pd

class TradingSimulator:
    def __init__(self, capital_inicial, imposto_curto=0.35, 
                 corretagem_por_acao=0.00, taxa_sec_r=0.0000229, 
                 taxa_taf_r=0.000145, limite_taf=7.27):
        self.capital_inicial = capital_inicial
        self.montante = capital_inicial
        self.quantidade_acoes = 0
        self.imposto_curto = imposto_curto
        self.corretagem_por_acao = corretagem_por_acao
        self.taxa_sec_r = taxa_sec_r
        self.taxa_taf_r = taxa_taf_r
        self.limite_taf = limite_taf
        self.transacoes = []
        self.posicoes = []
        self.total_corretagem = 0
        self.total_taxas = 0
        self.total_impostos = 0
        
    def executar_decisao(self, decisao, quantidade, preco, data):
        if decisao == 'compra':
            self._comprar(quantidade, preco, data)
        elif decisao == 'venda':
            self._vender(quantidade, preco, data)
        
        # Registrar estado atual
        self.posicoes.append({
            'data': data,
            'montante': self.montante,
            'quantidade_acoes': self.quantidade_acoes,
            'preco': preco,
            'valor_portfolio': self.montante + self.quantidade_acoes * preco
        })
    
    def _comprar(self, quantidade, preco, data):
        # Calcular custo total considerando comissões
        custo_total = quantidade * preco
        corretagem = quantidade * self.corretagem_por_acao
        
        if custo_total + corretagem <= self.montante:
            self.montante -= (custo_total + corretagem)
            self.quantidade_acoes += quantidade
            self.total_corretagem += corretagem
            
            self.transacoes.append({
                'data': data,
                'tipo': 'compra',
                'quantidade': quantidade,
                'preco': preco,
                'corretagem': corretagem,
                'total': custo_total + corretagem
            })
        else:
            print(f'Não foi possível comprar {quantidade} ações a {preco} em {data}.')
    
    def _vender(self, quantidade, preco, data):
        if quantidade <= self.quantidade_acoes:
            # Calcular receita e taxas
            valor_venda = quantidade * preco
            corretagem = quantidade * self.corretagem_por_acao
            
            # Calcular taxas SEC e TAF
            taxa_sec_aplicada = valor_venda * self.taxa_sec_r
            taxa_taf_aplicada = min(quantidade * self.taxa_taf_r, self.limite_taf)
            
            # Simplificação para imposto (na prática precisaria rastrear cada lote)
            # Aqui consideramos que todos os ganhos são de curto prazo
            cost_basis = self._calculate_cost_basis(quantidade)
            capital_gain = valor_venda - cost_basis
            tax = max(0, capital_gain * self.imposto_curto)
            
            # Atualizar posição e caixa
            net_proceeds = valor_venda - corretagem - taxa_sec_aplicada - taxa_taf_aplicada - tax
            self.montante += net_proceeds
            self.quantidade_acoes -= quantidade
            
            # Atualizar totais
            self.total_corretagem += corretagem
            self.total_taxas += (taxa_sec_aplicada + taxa_taf_aplicada)
            self.total_impostos += tax
            
            self.transacoes.append({
                'data': data,
                'tipo': 'venda',
                'quantidade': quantidade,
                'preco': preco,
                'corretagem': corretagem,
                'taxa_sec_aplicada': taxa_sec_aplicada,
                'taxa_taf_aplicada': taxa_taf_aplicada,
                'tax': tax,
                'net_proceeds': net_proceeds
            })
    
    def _calculate_cost_basis(self, quantidade):
        # Método simplificado - na implementação real seria necessário
        # rastrear cada compra individualmente (FIFO, LIFO ou específica)
        # Aqui usamos um preço médio dos ativos em carteira
        if not self.transacoes:
            return 0
        
        compras = [t for t in self.transacoes if t['tipo'] == 'compra']
        if not compras:
            return 0
        
        custo_quantidade_acoes = sum(t['quantidade'] for t in compras)
        custo_total = sum(t['quantidade'] * t['preco'] for t in compras)
        
        avg_cost = custo_total / custo_quantidade_acoes if custo_quantidade_acoes > 0 else 0
        return quantidade * avg_cost
    
    def obter_relatorio(self, final_preco):
        valor_portfolio = self.montante + self.quantidade_acoes * final_preco
        lucro_ou_prejuizo = valor_portfolio - self.capital_inicial
        return {
            'capital_inicial': self.capital_inicial,
            'final_valor_portfolio': valor_portfolio,
            'valor_portfolio_s_impostos': valor_portfolio + self.total_impostos, 
            'valor_portfolio_s_impostos_taxas': valor_portfolio + self.total_impostos + self.total_taxas,
            'quantidade_acoes_restantes': self.quantidade_acoes,
            'montante_restante': self.montante,
            'lucro_ou_prejuizo': lucro_ou_prejuizo,
            'retorno_percentual': (lucro_ou_prejuizo / self.capital_inicial) * 100 if self.capital_inicial > 0 else 0,
            'total_corretagem': self.total_corretagem,
            'total_taxas': self.total_taxas,
            'total_impostos': self.total_impostos,
            'custo_operacional_total': self.total_corretagem + self.total_taxas + self.total_impostos
        }

In [2]:
import pandas as pd
trade_df = pd.read_csv('stock_action_data.csv', parse_dates=['Fdate'])

# Inicializar com $10,000 de capital inicial
simulator = TradingSimulator(capital_inicial=10000)

# Processar decisões
for idx, row in trade_df.iterrows():
    acao = row['Action']
    preco = row['Current Price']
    data = row['Fdate']
    quantidade = 5  # 5 ações por transação
    
    simulator.executar_decisao(
        decisao=acao,
        quantidade=quantidade,
        preco=preco,
        data=data
    )

# Obter resumo final
resumo = simulator.obter_relatorio(trade_df['Current Price'].iloc[-1])
print(resumo)


{'capital_inicial': 10000, 'final_valor_portfolio': np.float64(10102.335164913937), 'valor_portfolio_s_impostos': np.float64(10134.445984449265), 'valor_portfolio_s_impostos_taxas': np.float64(10134.63809812446), 'quantidade_acoes_restantes': 30, 'montante_restante': 3051.2392465465896, 'lucro_ou_prejuizo': np.float64(102.33516491393675), 'retorno_percentual': np.float64(1.0233516491393675), 'total_corretagem': 0.0, 'total_taxas': 0.1921136751950819, 'total_impostos': 32.11081953532868, 'custo_operacional_total': 32.30293321052376}
